In [ ]:
import cv2
from ultralytics import YOLO
import json
from openai import OpenAI
import os
from dotenv import load_dotenv


load_dotenv()

class VideoProcessor:
    def __init__(self, model_path='yolov10n.pt'):
        self.model = YOLO(model_path)
        self.violations = []

    def process_video(self, video_path):
        cap = cv2.VideoCapture(video_path)
        frame_number = 0
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            results = self.model(frame)
            current_violations = self._analyze_frame(results, frame_number)
            self.violations.extend(current_violations)
            
            frame_number += 5
        
        cap.release()
        return self.violations

    def _analyze_frame(self, results, frame_number):
        persons = []
        hard_hats = []
        
        for result in results:
            for box in result.boxes:
                cls_id = int(box.cls)
                cls_name = self.model.names[cls_id]
                bbox = box.xyxy[0].tolist()
                
                if cls_name == 'person':
                    persons.append(bbox)
                elif cls_name == 'hardhat':
                    hard_hats.append(bbox)
        
        frame_violations = []
        for person in persons:
            if not self._has_hardhat(person, hard_hats):
                frame_violations.append({
                    'frame': frame_number,
                    'violation_type': 'no_hardhat',
                    'details': 'Person without hard hat detected'
                })
        
        return frame_violations

    def _has_hardhat(self, person_bbox, hard_hats):
        px1, py1, px2, py2 = person_bbox
        head_region = [px1, py1, px2, py1 + (py2 - py1) * 0.3]
        
        for hh in hard_hats:
            hx1, hy1, hx2, hy2 = hh
            if (hx1 < head_region[2] and hx2 > head_region[0] and
                hy1 < head_region[3] and hy2 > head_region[1]):
                return True
        return False

class ReportGenerator:
    def __init__(self):
        self.client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
    
    def generate_report(self, violations):
        summary = {
            'total_violations': len(violations),
            'common_violation_types': self._count_violation_types(violations),
            'example_frames': [v['frame'] for v in violations[:5]]
        }
        
        response = self.client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a safety compliance analyst. Generate a professional report based on the following data. Highlight key findings and recommend improvements."},
                {"role": "user", "content": f"Safety compliance data:\n{json.dumps(summary, indent=2)}"}
            ]
        )
        return response.choices[0].message.content
    
    def _count_violation_types(self, violations):
        counts = {}
        for v in violations:
            counts[v['violation_type']] = counts.get(v['violation_type'], 0) + 1
        return counts

if __name__ == "__main__":
    # Initialize components
    processor = VideoProcessor()#='path/to/custom_model.pt')  # Replace with safety-specific model
    report_gen = ReportGenerator()
    
    # Process video
    violations = processor.process_video('input_video.mp4')
    print(violations)
    # Generate and save report
    report = report_gen.generate_report(violations)
    with open('safety_report.md', 'w') as f:
        f.write(report)
    
    print(f"Report generated with {len(violations)} violations detected.")

In [ ]:
from inference_sdk import InferenceHTTPClient

CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",    api_key="API_KEY")

result = CLIENT.infer(your_image.jpg, model_id="ppe-detection-l80fg/4")

In [4]:
import pandas as pd
pd.DataFrame(violations).drop_duplicates()

,frame,violation_type,details
0,240,no_hardhat,Person without hard hat detected
1,865,no_hardhat,Person without hard hat detected
3,870,no_hardhat,Person without hard hat detected
5,875,no_hardhat,Person without hard hat detected
6,880,no_hardhat,Person without hard hat detected
...,...,...,...
848,4020,no_hardhat,Person without hard hat detected
849,4025,no_hardhat,Person without hard hat detected
850,4030,no_hardhat,Person without hard hat detected
851,4035,no_hardhat,Person without hard hat detected


In [8]:
violations[0]

{'frame': 240,
 'violation_type': 'no_hardhat',
 'details': 'Person without hard hat detected'}

In [ ]:
pai.api_key.set(os.getenv('PANDAS_AI_KEY'))